In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
%matplotlib inline

from numpy.fft import fft, rfft, rfftfreq, irfft
import seaborn as sns

FD = 2000 # 2кГц - частота дискретизации сигнала

In [25]:
signal = readSignalFromFile("kdms/6.txt").sig1

In [7]:
def my_work(signal, lock, storage):
    mn, mx = getExtremaSuperFast(signal)
    with lock:
        storage.append((signal, mn, mx))

def parallel_box(work, files):
    from threading import Thread, Lock
    storage = []
    workers = []
    lock = Lock()
    for file_name in files:
        p = Thread(target=my_work, args=(file_name, lock, storage))
        p.start()
        workers.append(p)
    
    for worker in workers:
        worker.join()
    
    with lock:
        return storage

In [109]:
from multiprocessing import Pool

def poolTest(threads_num, files):
    pool = Pool(threads_num)
    pool.map(my_work, files)
    pool.close()
    pool.join()

In [8]:
file_names = ["kdms/"+str(i)+".txt" for i in range(1, 8)]
files = [readSignalFromFile(file_name).sig1 for file_name in file_names]
%timeit signals = parallel_box(my_work, files)

1 loops, best of 3: 1min 10s per loop


In [107]:
file_names = ["kdms/"+str(i)+".txt" for i in range(1, 8)]
files = [readSignalFromFile(file_name).sig1 for file_name in file_names]
def asUsual():
    storage = []
    for file_name in files:
        mn, mx = getExtremaSuperFast(file_name)
        storage.append((signal, mn, mx))
%timeit asUsual()

1 loops, best of 3: 23.8 s per loop


In [ ]:
file_names = ["kdms/"+str(i)+".txt" for i in range(1, 8)]
files = [readSignalFromFile(file_name).sig1 for file_name in file_names]
%timeit poolTest(8, files)

# Инструментарий: используемые функции

In [5]:
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
from numpy.fft import fft, rfft, rfftfreq, irfft

def drawSignalMxMn(sig, extrema=None):
    mn, mx = getExtremaSuperFast(sig) if extrema == None else extrema
    plt.figure(figsize=(18, 6))
    plt.plot(sig)
    plt.scatter(mx, [sig[x] for x in mx], c='r', s=70)
    plt.scatter(mx, [sig[x] for x in mn], c='g', s=70)

def readSignalFromFile(file_name):
    splitted = [s.split() for s in open(file_name, 'r').readlines()]
    return pd.DataFrame(splitted, columns=['sig1', 'sig2']).astype(float)

def getDefaultPeak():
    import pickle
    return pickle.load(open('patterns/default_QRS.dump', 'r'))

def getExtrema(signal, peak_criterion=0.7, epsilon=10, model=None):
    from sklearn.preprocessing import scale
    
    if (model == None):
        model = getDefaultPeak() 
    scaled = scale(model, with_std=False)
    ideal_max = np.argmax(scaled)
    ideal_min = np.argmin(scaled)
    
    win_len = len(model)
    corr = []
    for i in range(0, len(signal) - win_len):
        win = signal[i:i+win_len]
        corr.append(np.correlate(scaled, scale(win, with_std = False)))

    m = max(corr)
    peaks = np.array([x if x > m*peak_criterion else 0 for x in corr]);
    
    from scipy.signal import argrelextrema
    max_peak_points = argrelextrema(peaks, np.greater) + ideal_max
    min_peak_points = argrelextrema(peaks, np.greater) + ideal_min
    
    real_max_peak_points = [np.argmax(signal[x-epsilon : x+epsilon]) - signal.index[0] for x in max_peak_points[0]]
    real_min_peak_points = [np.argmin(signal[x-epsilon : x+epsilon]) - signal.index[0] for x in min_peak_points[0]]
    
    return real_min_peak_points, real_max_peak_points

def analyzeLiterals(s, mn, mx):
    periods = []
    delta_periods = []
    amplitudes = []#[s[mx[0]] - s[mn[0]]]
    delta_amplitudes = []
    angles = []
    delta_angles = []
    abs_deltas = []
    for i in range(len(mx)-1):
        p = mx[i+1]-mx[i]
        periods.append(np.abs(p))
        am = np.abs(s[mx[i]] - s[mn[i]])
        amplitudes.append(am)
        angles.append(float(am)/p)
        if (i >= 1):
            delta_periods.append((periods[i] - periods[i-1])>=0)
            delta_amplitudes.append((amplitudes[i]-amplitudes[i-1])>=0)
            abs_deltas.append(np.abs(amplitudes[i]-amplitudes[i-1]))
            delta_angles.append((angles[i]-angles[i-1])>=0)
    deltas = zip(delta_amplitudes, delta_periods, delta_angles)
    
    for i in range(len(deltas)):
        if ((deltas[i] == (True, False, False)) | (deltas[i]==(False, True, True))):
            print deltas[i]
            print amplitudes[i-1], amplitudes[i]
            print periods[i-1], periods[i]
            print angles[i-1], angles[i]
    
    liters_dict = {
        (True, True, True) : "A",
        (False, False, True) : "B",
        (True, False, True) : "C",
        (False, True, False) : "D",
        (True, True, False) : "E",
        (False, False, False) : "F",
        (True, False, False) : "X"
    }

    literals = [liters_dict[c] for c in deltas]
    
    from collections import Counter
    codograms = Counter()
    for i in range(len(literals)-3):
        win = literals[i:i+3]
        code = ''.join(win)
        codograms[code] += 1
    return codograms, literals, abs_deltas

def getMids(points):
    prev= points[0]
    mids = []
    for i in points[1:]:
        center = (i + prev)/2
        mids.append(center)
        prev = i
    return mids

def deSocket(sig, fd, interval=(40, 60)):
    from numpy.fft import rfft, rfftfreq
    
    #mn, mx = getExtrema(sig, model)
    
    spectrum = rfft(sig)
    x_axis = rfftfreq(len(sig), 1./fd)
    a,b = hz(len(sig), fd, interval[0], interval[1])
    w = np.argmax(np.abs(spectrum[a:b])) #частота 
    A = spectrum[a+w]/len(sig) #амплитутда
    
    def cos_to_remove(x):
        return (2.)*A*np.exp(2*1j*np.pi*(a+w)*(x-len(sig)/2)/len(sig))

    cos_graph = [cos_to_remove(x) for x in range(len(sig))]
    return cos_graph + sig


def deSocketFull(sig, fd, interval=(40, 60), delta=517, border=50):
    from numpy.fft import rfft, rfftfreq
    
    clr_sig = sig.copy()
    
    mn, mx = getExtrema(clr_sig)
    mids1 = getMids(mx)
    mx1 = sorted(np.concatenate([mids1, mx]))
    mids = getMids(mx1)
    for i in range(len(mids)):
        sig_part = clr_sig[mids[i]-delta/2 : mids[i]+delta/2]
        spectrum = rfft(sig_part)
        x_axis = rfftfreq(len(sig_part), 1./fd)
        a,b = hz(len(sig_part), fd, interval[0], interval[1])
        w = np.argmax(np.abs(spectrum[a:b])) #частота 
        A = spectrum[a+w]/len(sig_part) #амплитутда

        def cos_to_remove(x):
            return (2.)*A*np.exp(2*1j*np.pi*(a+w)*(x)/len(sig_part))

        left = mids[i]-delta/2-200
        right = mids[i]+delta/2+200
        #cos_graph = [cos_to_remove(x) for x in range(len(sig[mx[i]:mx[i+1]]) + 2*border)]
        cos_graph = [cos_to_remove(x) for x in range(-(right-left)/2, (right-left)/2)]
        
        graph_len = len(cos_graph) -1
        
        scale = border
        for j in range(scale):
            cos_graph[j] *= float(j)/(scale)
            cos_graph[graph_len-j] *= float(j)/(scale)
        
        clr_sig[left:right] += np.real(cos_graph)
        start_index = clr_sig.index[0]
    return clr_sig, mn + start_index, mx + start_index

def getExtremaSuperFast(signal2, scan_area = 700, sdf_area = 6000, abs_indexes=False):
    from sklearn.preprocessing import scale
    mn, mx = getExtrema(signal2[:sdf_area])
    signal = signal2.values
    
    delta_max = []
    for i in range(1, len(mx)):
        delta_max.append(mx[i] - mx[i-1])
    
    delta_min = []
    for i in range(1, len(mn)):
        delta_min.append(mn[i] - mn[i-1])
        
    mean_delta_max = int(np.mean(delta_max))
    mean_delta_min = int(np.mean(delta_min))
    
    max_center =  mx[-1] + mean_delta_max
    min_center =  mn[-1] + mean_delta_min
    while ((max_center < len(signal)) & (min_center < len(signal))):
        current_max = max_center - scan_area/2 + np.argmax(scale(signal[max_center - scan_area/2 : max_center + scan_area/2], with_std = False))
        current_min = min_center - scan_area/2 + np.argmin(scale(signal[min_center - scan_area/2 : min_center + scan_area/2], with_std = False))
        delta_max = delta_max[1:]
        delta_max.append(current_max - mx[-1])
        delta_min = delta_min[1:]
        delta_min.append(current_min - mn[-1])
        #mean_delta_max = int(np.mean([mean_delta_max, current_max - mx[-1]]))
        #mean_delta_min = int(np.mean([mean_delta_min, current_min - mn[-1]]))
        mx.append(current_max)
        mn.append(current_min)
        max_center = current_max + int(np.mean(delta_max))
        min_center = current_min + int(np.mean(delta_min))
        #max_center =  current_max + mean_delta_max
        #min_center =  current_min + mean_delta_min
    
    
    mx = [i + signal2.index[0] for i in mx]
    mn = [i + signal2.index[0] for i in mn]
    return mn, mx